In [1]:
from Helmholtz_recon import *
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import cv2
from scipy import ndimage
import os
import scipy
import pandas as pd


In [7]:
# MRI scan data 
path = 'C:\\Users\\20203226\\Documents\\GitHub\\BEP\\MRI results\\2_TSE_tra_A.mat'
ph_b = scipy.io.loadmat(path)['SE']

In [13]:
# b1p = b1p * np.exp(1j*ph_b)
# b1p = b1p[:,:,80]

# sig_HH = Helmholtz(b1p, ks=7)
sample = 99
new_dir = path = os.path.join('C:\\Users\\20203226\\Documents\\GitHub\\BEP\\MRI results\\masks\\Round_2_TSE\\', 'Sample_{}'.format(sample))
os.mkdir(new_dir)

for slice in range(1,4):

    sig_HH = Helmholtz_ph(np.angle(ph_b[:,:,slice]))
    sig_b = smooth_HH(sig_HH, 3)

    get_ipython().run_line_magic('matplotlib', 'qt')

    #set up the overarching window
    fig, axes = plt.subplots(1,1)
    fig.figsize = [16,16]
    
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()

    #plot the images
    axes.imshow(sig_b, vmin=0, vmax=6)
    axes.set_title("Phase image smoothed")

    #accumulate points
    points = plt.ginput(n=-1, timeout=30)
    plt.close(fig)

    #restore to inline figure placement
    get_ipython().run_line_magic('matplotlib', 'inline')
    
    mask = np.zeros(sig_b.shape, dtype="uint8")
    
    if len(points)>0:
        middle_x = (int(points[0][0]) + int(points[1][0]))// 2
        middle_y = (int(points[0][1]) + int(points[1][1]))// 2
        radius = int(np.sqrt(((points[0][0]- points[1][0])**2) + ((points[0][1]- points[1][1])**2)) / 2)
        cv2.circle(mask, (middle_x, middle_y), radius, 255, -1)

    cv2.imwrite('C:\\Users\\20203226\\Documents\\GitHub\\BEP\\MRI results\\masks\\Round_2_TSE\\Sample_{}\\'.format(sample) + '{}.png'.format(slice), mask)
    
